In [1]:
from __future__ import absolute_import, division
from __future__ import print_function, unicode_literals
import pints.toy as toy
import pints
import numpy as np
import logging
import math
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from numpy import inf
import copy 
import pickle
import time
import CMA as CMA

In [9]:
def CMA_on_HyperRect_Homogeneous(opt, 
                 problem,
                 sampler,
                 sampler_x0, 
                 n_chains=1,
                 chain_size=100,
                 need_sensitivities=False, 
                eval_fun=['ESS']):
    
    optimizer_best_fxs = []
    optimizer_best_xs = []
    
    optimizer_best_fx = np.inf
    optimizer_best_x = 0
    
    for _ in range(50):
        
        # Getting the samples of hyper-parameters for the samplers
        optimizer_xs = opt.ask()
        
        # Saving the score of each sample of hyper-parameters
        optimizer_fxs = []
        
        # Evaluate performance for each hyper-parameter configuration
        for x in optimizer_xs:
            
            our_x = [[x[0], x[1]], False]
            
            # Initialise function evaluations and matrix for chains
            function_evaluations = 0 
            chains = []
            
            # Return array of samples for each chain
            for i in range(n_chains):

                # Create sampler object and set hyperparameter
                curr_x0 = sampler_x0[i]
                mcmc = sampler(curr_x0)
                mcmc.set_hyper_parameters(our_x)
                i_chain = []
                
                # Update until we have ``chain_size`` samples
                # Update function evaluations each time we use ask(),tell()
                while len(i_chain) < chain_size:
                    x = mcmc.ask()
                    if need_sensitivities:
                        fx, grad = problem.evaluateS1(x)
                        sample = mcmc.tell((fx, grad))
                    else:
                        fx = problem.evaluateS1(x)
                        sample = mcmc.tell(fx)
                    function_evaluations += 1
                    if sample is not None:
                        i_chain.append(sample)
                
                # Append ith chain to list of chains       
                chains.append(i_chain)
                
            chains = np.array(chains, copy=True)
            optimizer_fx = 0
            # Calculate the score of the sampler with the given hyper-parameters
            # Get the KL if requested
            if 'KL' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                optimizer_fx = kl/len(chains)                
            
            elif 'KL-ITER' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                avg_kl = kl/len(chains)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = avg_kl * avg_iteration_count
            
            # Get the ESS if requested
            elif 'ESS' in eval_fun:
                ess = np.zeros(chains[0].shape[1])
                for chain in chains:
                    ess += np.array(pints._diagnostics.effective_sample_size(chain))
                ess /= len(chains)
                ess = np.min(ess)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = math.sqrt(avg_iteration_count) / ess
                
            optimizer_fxs.append(optimizer_fx)
        opt.tell(optimizer_fxs)
        
        optimizer_best_fxs.append(opt.fbest())
        optimizer_best_xs.append(opt.xbest())
        
        if opt.fbest() < optimizer_best_fx:
            optimizer_best_fx = opt.fbest()
            optimizer_best_x = opt.xbest()
    
    print(optimizer_best_fx)
    print(optimizer_best_x)
    print()
    
    return optimizer_best_fxs, optimizer_best_xs

In [10]:
rosenbrock = pints.toy.RosenbrockLogPDF()
eggshape = pints.toy.SimpleEggBoxLogPDF(2,4)
Logistic = pints.toy.LogisticModel()

problem = eggshape

# Define the starting points of the samplers
sampler_rosenbrock_x0 = xs = [
    [0,0],
    [3,8],
    [2,4],
]

sampler_egg_x0 = xs = [
    [-10,-10],
    [0,0],
    [10,10],
]

sampler_x0 = sampler_egg_x0

# Define the starting point of the optimzer
dummy = pints.SliceHyperrectanglesMCMC([2, 4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1]]

boundary = pints.RectangularBoundaries([0,0],[20,20])

sampler = pints.SliceHyperrectanglesMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_HyperRect_Homogeneous(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

0.6954018606609961
[9.21391654 9.02364727]

0.6797860425897491
[11.36316537  9.10693217]

0.6864313595714472
[9.56691377 9.44077568]

0.696544483132432
[12.29495318 11.64797848]

0.658750850245878
[9.16495498 9.71272276]



## Results for the Rosenbrock Problem using different Loss 

In [ ]:
# Avg_iteration count over ESS
39.22133186462336
[0.54406868 0.87978578]
35.19997635159976
[0.65310227 0.96749411]
19.32533975593854
[0.02304884 0.09199689]
25.59287940905728
[0.00087682 0.18857804]
40.2368614691648
[0.99958012 0.58758153]

In [ ]:
# 1 over ESS
0.060240947544655324
[6.88960141 7.87139767]

0.05855631030168425
[11.34852473 19.34966525]

0.05478297275859089
[14.40292733 12.15825586]

0.05240072296170241
[14.78413561 16.09348001]

0.060761246693836375
[14.07404409  9.92222866]

In [ ]:
# sqrt(avg_ieration) over ESS
0.6579954014060239
[9.44235379 8.82797313]

0.6747332577632024
[10.04051936  9.21943616]

0.6754667829729044
[8.29234221 8.26721892]

0.693838312255056
[8.15209441 7.4049454 ]

0.6733647984144733
[8.86375154 9.24731263]

## Results for the egg Problem

In [ ]:
0.6954018606609961
[9.21391654 9.02364727]

0.6797860425897491
[11.36316537  9.10693217]

0.6864313595714472
[9.56691377 9.44077568]

0.696544483132432
[12.29495318 11.64797848]

0.658750850245878
[9.16495498 9.71272276]

## Logistic Model Problem


In [3]:
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

class ExampleModel(pints.ForwardModel):
    
    def __init__(self):
        self.model = toy.LogisticModel()
    def simulate(self, x, times):
        return self.model.simulate([x[0]/1000, x[1]], times)
    def simulateS1(self, x, times):
        values, gradient = self.model.simulateS1([x[0]/1000, x[1]], times)
        return values, gradient
    def n_parameters(self):
        return 2

# Then create an instance of our new model class
model = ExampleModel()

In [4]:
# Create some toy data
real_parameters = [0.015*10000, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.GaussianLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01*10000, 400, noise * 0.1],
    [0.02*10000, 600, noise * 100],
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
num_chains = 3
sampler_logistic_x0 = [
    real_parameters * 1.01,
    real_parameters * 0.99,
    real_parameters * 1.01
]

In [7]:
problem = log_posterior

sampler_x0 = sampler_logistic_x0

# Define the starting point of the optimzer
dummy = pints.SliceHyperrectanglesMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2]]

boundary = pints.RectangularBoundaries([0,0,0],[20,20,20])

sampler = pints.SliceHyperrectanglesMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_HyperRect_Homogeneous(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

1.171352545031565
[16.80640373 15.0343616   7.34175251]

1.1215557984532354
[4.92052093 6.28257363 2.88642057]

1.1720671160551777
[12.00490911 10.0007691   6.4452416 ]

1.0858744291049671
[6.39426966 6.9034228  2.79639242]

1.0997394181683062
[6.64677631 5.13585011 4.18637856]



In [ ]:
1.171352545031565
[16.80640373 15.0343616   7.34175251]

1.1215557984532354
[4.92052093 6.28257363 2.88642057]

1.1720671160551777
[12.00490911 10.0007691   6.4452416 ]

1.0858744291049671
[6.39426966 6.9034228  2.79639242]

1.0997394181683062
[6.64677631 5.13585011 4.18637856]